In [41]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.linear_model import LogisticRegression
print("TF  Version",tf.__version__)

TF  Version 2.2.0


In [42]:
# check and set path before loading modules
INPUT_DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
OUTPUT_DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
if os.getcwd() != OUTPUT_DIR:
    os.chdir(OUTPUT_DIR)
    
import functions_model_definition as md
import functions_read_data as rdat

In [44]:
# Define Version
version = "CIBLSX" # one of:
#version = "CIB" # one of:

# Define Model Version
model_version = 4

# Select naming convention (for CIBLSX model_version >= 3 should be False)
comp_mode = False # if True: use old naming convention

# define paths
DATA_DIR, WEIGHT_DIR, DATA_OUTPUT_DIR, PIC_OUTPUT_DIR, pic_save_name = rdat.dir_setup(
    INPUT_DIR, OUTPUT_DIR, version, model_version, 
    compatibility_mode=comp_mode)

save_csv = False

In [45]:
## load images and ids
(X_in, pat_ids, id_tab, all_results_tab, pat_orig_tab, pat_norm_tab, num_models) = rdat.version_setup(
    DATA_DIR = DATA_DIR, 
    version = version, 
    model_version = model_version,
    compatibility_mode=comp_mode)

Results Table does not exist for CIBLSX M4. Returning None for all_results_tab.


In [46]:
pat_norm_tab

,p_id,age,sexm,nihss_baseline,stroke_beforey,tia_beforey,rf_hypertoniay,rf_diabetesy,rf_hypercholesterolemiay,rf_smokery,rf_atrial_fibrillationy,rf_chdy,mrs_before_1,mrs_before_2,mrs_before_3,mrs_before_4
0,1,-2.196946,1,-0.657875,0,0,0,0,1,0,0,0,0,0,0,0
1,2,-1.407829,1,0.482720,0,0,0,0,1,0,0,0,0,0,0,0
2,3,0.893762,1,1.786257,0,0,1,1,1,1,1,0,0,0,0,0
3,5,0.828002,1,0.482720,0,0,0,1,0,0,1,0,1,0,0,0
4,6,1.222560,0,0.156836,0,0,1,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,555,0.900338,1,-0.494933,0,0,0,0,1,0,1,1,1,0,0,0
503,556,-1.684020,1,-0.820817,0,0,1,0,0,1,0,0,0,0,0,0
504,557,0.334804,0,-0.657875,0,0,0,0,1,0,0,0,0,0,0,0
505,559,0.347956,0,-0.494933,0,0,1,0,1,0,0,0,0,0,1,0


In [48]:
data_split = rdat.split_data(id_tab, X_in, 0, X_tab = pat_norm_tab)


In [52]:
model = LogisticRegression(penalty='none')
model.fit(data_split["X_tab"]["train"] , data_split["y"]["train"])

LogisticRegression(penalty='none')

In [53]:
feature_names = pat_norm_tab.columns[1:16]
coefficients = model.coef_[0] 
coefficients_dict = dict(zip(feature_names, coefficients))
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
print("Intercept:", model.intercept_)
print("************************************************************")
print(coefficients_df)


Intercept: [-2.92356883]
************************************************************
                     Feature  Coefficient
0                        age     0.380409
1                       sexm     0.711133
2             nihss_baseline     1.056050
3             stroke_beforey     0.558557
4                tia_beforey    -0.086767
5             rf_hypertoniay     0.258402
6               rf_diabetesy    -0.897642
7   rf_hypercholesterolemiay    -0.154534
8                 rf_smokery     0.334722
9    rf_atrial_fibrillationy     0.575728
10                   rf_chdy     0.147304
11              mrs_before_1     0.455729
12              mrs_before_2     1.600805
13              mrs_before_3     1.736446
14              mrs_before_4    16.871087


# Save split to test in R

In [36]:
df = pd.DataFrame(data_split["X_tab"]["train"])
df.columns = pat_norm_tab.columns[1:16]
df["binarymRS"] = data_split["y"]["train"]

In [39]:
df.to_csv("/tf/notebooks/schnemau/xAI_stroke_3d/graveyard code/data_frame_logistic.csv",  index=False) 